In [1]:
import os
import json
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from nn.models import OneHiddenMLP
from nn.training import train_passive, TrainConfig
from nn.evaluation import evaluate_classification
from nn.experiments import ActiveConfig, run_active_classification
from nn.strategies import uncertainty_sampling, sensitivity_sampling, UncertaintySamplingConfig
from typing import Dict

SAVE_DIR = os.path.join('..', 'report', 'figures')
DATA_DIR = os.path.join('..', 'data')
os.makedirs(SAVE_DIR, exist_ok=True)

with open(os.path.join(DATA_DIR, 'cls_uncertainty_results.json'), 'r') as f:
    unc_results = json.load(f)
with open(os.path.join(DATA_DIR, 'cls_sensitivity_results.json'), 'r') as f:
    sen_results = json.load(f)
with open(os.path.join(DATA_DIR, 'passive_cls_best.json'), 'r') as f:
    pas_results = json.load(f)

DATASETS = ['iris', 'wine', 'breast_cancer']
METRICS = ['accuracy', 'f1_macro']
UNCERTAINTY_METHODS = ['entropy', 'margin', 'least_confidence']

ACTIVE_PARAMS = {
    'iris': {'init': 10, 'query': 5, 'budget': 80},
    'wine': {'init': 10, 'query': 5, 'budget': 100},
    'breast_cancer': {'init': 20, 'query': 10, 'budget': 200},
}

In [ ]:
def get_data_splits(dataset: str):
    # Load data
    if dataset == "iris":
        ds = datasets.load_iris()
    elif dataset == "wine":
        ds = datasets.load_wine()
    elif dataset == "breast_cancer":
        ds = datasets.load_breast_cancer()
    
    X, y = ds.data, ds.target

    X_train_val, X_test, y_train_val, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    
    return X_train_val, X_test, y_train_val, y_test

def evaluate_passive_test(dataset: str) -> Dict[str, float]:
    # Get best hyperparameters from tuning results
    best_cfg = pas_results[dataset]['best_cfg']
    lr = best_cfg['lr']
    wd = best_cfg['wd']
    hidden = best_cfg['hidden']
    bs = best_cfg['bs']
    
    # Get data splits
    X_train_val, X_test, y_train_val, y_test = get_data_splits(dataset)
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_val)
    X_test_scaled = scaler.transform(X_test)
    
    # Convert to tensors
    X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train_val, dtype=torch.long)
    X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long)
    
    # Create datasets
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    
    train_loader = DataLoader(train_dataset, batch_size=bs, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=bs, shuffle=False)
    
    # Train model
    model = OneHiddenMLP(input_dim=X_train_scaled.shape[1], hidden_units=hidden, output_dim=len(np.unique(y_train_val)))
    loss_fn = nn.CrossEntropyLoss()
    config = TrainConfig(learning_rate=lr, weight_decay=wd, batch_size=bs, max_epochs=200, patience=20)
    
    train_passive(model, train_loader, test_loader, loss_fn, config)
    
    # Evaluate on test set
    metrics = evaluate_classification(model, test_loader)
    
    return metrics

def evaluate_active_test(dataset: str, strategy: str, method: str, budget: int) -> Dict[str, float]:
    dataset_params = ACTIVE_PARAMS[dataset]
    init = dataset_params['init']
    query = dataset_params['query']
    hidden, bs = 64, 64
    if strategy == 'uncertainty':
        best_cfg = unc_results[dataset][method]['best_cfg']['train_config']
        lr = best_cfg['learning_rate']
        wd = best_cfg['weight_decay']
    else:
        best_cfg = sen_results[dataset]['best_cfg']['train_config']
        lr = best_cfg['learning_rate']
        wd = best_cfg['weight_decay']

    # Get data splits
    X_train_val, X_test, y_train_val, y_test = get_data_splits(dataset)
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_val)
    X_test_scaled = scaler.transform(X_test)
    
    # Convert to tensors
    X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train_val, dtype=torch.long)
    X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long)

    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

    test_loader = DataLoader(test_dataset, batch_size=bs, shuffle=False)
    
    train_config = TrainConfig(learning_rate=lr, weight_decay=wd, batch_size=bs, 
                                max_epochs=200, patience=20)
    
    # Create initial labeled pool
    num_train = X_train_scaled.shape[0]
    labeled_indices = torch.randperm(num_train)[:init]
    unlabeled_indices = torch.tensor([i for i in range(num_train) if i not in labeled_indices.tolist()], dtype=torch.long)
    
    x_pool = torch.tensor(X_train_scaled, dtype=torch.float32)
    y_pool = y_train_tensor.clone()
    
    # Active learning loop
    while labeled_indices.numel() < min(budget, num_train):
        # Train model on current labeled set
        train_subset = TensorDataset(x_pool[labeled_indices], y_pool[labeled_indices])
        
        train_loader = DataLoader(train_subset, batch_size=bs, shuffle=True)
        
        model = OneHiddenMLP(input_dim=X_train_scaled.shape[1], hidden_units=hidden, output_dim=len(np.unique(y_train_val)))
        loss_fn = nn.CrossEntropyLoss()
        
        train_passive(model, train_loader, test_loader, loss_fn, train_config)
        
        if unlabeled_indices.numel() == 0:
            break
        
        # Query selection
        if strategy == 'uncertainty':
            sel = uncertainty_sampling(
                model,
                x_pool[unlabeled_indices].to('cpu'),
                query,
                UncertaintySamplingConfig(mode="classification", method=method),
            )
        elif strategy == 'sensitivity':
            sel = sensitivity_sampling(model, x_pool[unlabeled_indices].to('cpu'), query)
        
        # Update labeled and unlabeled sets
        newly_selected = unlabeled_indices[sel]
        labeled_indices = torch.unique(torch.cat([labeled_indices, newly_selected]))
        mask = torch.ones_like(unlabeled_indices, dtype=torch.bool)
        mask[sel] = False
        unlabeled_indices = unlabeled_indices[mask]
        
        if labeled_indices.numel() >= budget:
            break
    
    # Final evaluation on test set
    final_train_subset = TensorDataset(x_pool[labeled_indices], y_pool[labeled_indices])
    final_train_loader = DataLoader(final_train_subset, batch_size=bs, shuffle=True)
    final_test_loader = DataLoader(test_dataset, batch_size=bs, shuffle=False)
    
    final_model = OneHiddenMLP(input_dim=X_train_scaled.shape[1], hidden_units=hidden, output_dim=len(np.unique(y_train_val)))
    loss_fn = nn.CrossEntropyLoss()
    
    train_passive(final_model, final_train_loader, final_test_loader, loss_fn, train_config)
    
    # Evaluate on test set
    metrics = evaluate_classification(final_model, final_test_loader)
    
    return metrics


In [3]:
test_results = {
    'passive': {},
    'uncertainty': {},
    'sensitivity': {}
}

for dataset in DATASETS:
    print(f"Running passive on {dataset}...")
    test_results['passive'][dataset] = evaluate_passive_test(dataset)

# Evaluate uncertainty-based active learning on test set
for dataset in DATASETS:
    budget = ACTIVE_PARAMS[dataset]['budget']
    test_results['uncertainty'][dataset] = {}
    for method in UNCERTAINTY_METHODS:
        print(f"Running uncertainty on {dataset} - {method}...")
        test_results['uncertainty'][dataset][method] = {}
        test_results['uncertainty'][dataset][method][str(budget)] = evaluate_active_test(dataset, 'uncertainty', method, budget)

# Evaluate sensitivity-based active learning on test set
for dataset in DATASETS:
    budget = ACTIVE_PARAMS[dataset]['budget']
    print(f"Running sensitivity on {dataset}...")
    test_results['sensitivity'][dataset] = {}
    test_results['sensitivity'][dataset][str(budget)] = evaluate_active_test(dataset, 'sensitivity', '', budget)


Running passive on iris...
Running passive on wine...
Running passive on breast_cancer...
Running uncertainty on iris - entropy...
Running uncertainty on iris - margin...
Running uncertainty on iris - least_confidence...
Running uncertainty on wine - entropy...
Running uncertainty on wine - margin...
Running uncertainty on wine - least_confidence...
Running uncertainty on breast_cancer - entropy...
Running uncertainty on breast_cancer - margin...
Running uncertainty on breast_cancer - least_confidence...
Running sensitivity on iris...
Running sensitivity on wine...
Running sensitivity on breast_cancer...


In [5]:
summary_data = []

for dataset in DATASETS:
    budget = ACTIVE_PARAMS[dataset]['budget']
    # Passive learning
    summary_data.append({
        'dataset': dataset,
        'method': 'passive',
        'budget': budget,
        'accuracy': test_results['passive'][dataset]['accuracy'],
        'f1': test_results['passive'][dataset]['f1_macro'],
    })
    
    # Uncertainty methods
    for method in UNCERTAINTY_METHODS:
        max_budget = str(budget)
        summary_data.append({
            'dataset': dataset,
            'method': f'uncertainty_{method}',
            'budget': budget,
            'accuracy': test_results['uncertainty'][dataset][method][max_budget]['accuracy'],
            'f1': test_results['uncertainty'][dataset][method][max_budget]['f1_macro'],
        })
    
    # Sensitivity method
    max_budget = str(budget)
    summary_data.append({
        'dataset': dataset,
        'method': 'sensitivity',
        'budget': budget,
        'accuracy': test_results['sensitivity'][dataset][max_budget]['accuracy'],
        'f1': test_results['sensitivity'][dataset][max_budget]['f1_macro'],
    })

# Convert to DataFrame for nice display
df = pd.DataFrame(summary_data)
print(df.round(4))

# Save summary
df.to_csv(os.path.join(SAVE_DIR, 'cls_comparison_summary_test.csv'), index=False)

          dataset                        method  budget  accuracy      f1
0            iris                       passive      80    0.9667  0.9666
1            iris           uncertainty_entropy      80    0.9333  0.9333
2            iris            uncertainty_margin      80    0.9667  0.9666
3            iris  uncertainty_least_confidence      80    0.9333  0.9333
4            iris                   sensitivity      80    0.9667  0.9666
5            wine                       passive     100    0.9722  0.9710
6            wine           uncertainty_entropy     100    0.9722  0.9710
7            wine            uncertainty_margin     100    0.9722  0.9710
8            wine  uncertainty_least_confidence     100    0.9722  0.9710
9            wine                   sensitivity     100    0.9444  0.9407
10  breast_cancer                       passive     200    0.9649  0.9623
11  breast_cancer           uncertainty_entropy     200    0.9737  0.9719
12  breast_cancer            uncertain